In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler 
from sklearn.model_selection import train_test_split

Only column C-M needs to be imported

In [2]:
dataset = pd.read_csv('Churn-Rate.csv',usecols=range(2,13), sep=';', decimal=',')
dataset


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619.0,France,Female,42,2.0,0.00,1.0,1.0,1.0,101348.88,Yes
1,608.0,Spain,Female,41,1.0,83807.86,1.0,0.0,1.0,112542.58,No
2,502.0,France,Female,42,8.0,159660.80,3.0,1.0,0.0,113931.57,Yes
3,699.0,France,Female,39,1.0,0.00,2.0,0.0,0.0,93826.63,No
4,850.0,Spain,Female,43,2.0,125510.82,1.0,1.0,1.0,79084.10,No
...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
dataset


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619.0,France,Female,42,2.0,0.00,1.0,1.0,1.0,101348.88,Yes
1,608.0,Spain,Female,41,1.0,83807.86,1.0,0.0,1.0,112542.58,No
2,502.0,France,Female,42,8.0,159660.80,3.0,1.0,0.0,113931.57,Yes
3,699.0,France,Female,39,1.0,0.00,2.0,0.0,0.0,93826.63,No
4,850.0,Spain,Female,43,2.0,125510.82,1.0,1.0,1.0,79084.10,No
...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dataset.dtypes

CreditScore        float64
Geography           object
Gender              object
Age                 object
Tenure             float64
Balance            float64
NumOfProducts      float64
HasCrCard          float64
IsActiveMember     float64
EstimatedSalary    float64
Exited              object
dtype: object

Lines without CreditScore must be deleted

`copy` damit keien Seiteneffekte entstehen können (write on copy)

In [5]:
dataset = dataset.dropna(subset='CreditScore').copy(deep=False)
dataset


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619.0,France,Female,42,2.0,0.00,1.0,1.0,1.0,101348.88,Yes
1,608.0,Spain,Female,41,1.0,83807.86,1.0,0.0,1.0,112542.58,No
2,502.0,France,Female,42,8.0,159660.80,3.0,1.0,0.0,113931.57,Yes
3,699.0,France,Female,39,1.0,0.00,2.0,0.0,0.0,93826.63,No
4,850.0,Spain,Female,43,2.0,125510.82,1.0,1.0,1.0,79084.10,No
...,...,...,...,...,...,...,...,...,...,...,...
995,610.0,Spain,Male,29,9.0,0.00,3.0,0.0,1.0,83912.24,No
996,811.0,Spain,Male,44,3.0,0.00,2.0,0.0,1.0,78439.73,No
997,587.0,Spain,Male,62,7.0,121286.27,1.0,0.0,1.0,6776.92,No
998,811.0,Germany,Female,28,4.0,167738.82,2.0,1.0,1.0,9903.42,No


Fix datatype of columns

In [6]:
dataset[['NumOfProducts', 'HasCrCard', 'IsActiveMember', 'CreditScore']
        ] = dataset.loc[:, ['NumOfProducts', 'HasCrCard', 'IsActiveMember', 'CreditScore']].astype(int)


Convert Value wich are out of range or faulty

In [7]:
age_numeric = pd.to_numeric(dataset.loc[:, 'Age'], errors='coerce')

age_fault = age_numeric.replace(0,np.NaN)



dataset['Age'] = age_fault



Check for values

In [8]:
dataset[dataset.isnull().any(axis=1)].style.applymap(
    lambda x: 'background:coral' if pd.isna(x) else '')


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
12,476,France,Female,nan,10.000000,0.000000,2,1,0,26260.980000,No
22,510,Spain,nan,38.000000,4.000000,0.000000,1,1,0,118913.530000,Yes
24,846,France,Female,nan,5.000000,0.000000,1,1,1,187616.160000,No
58,511,Spain,Female,nan,4.000000,0.000000,1,1,0,1643.110000,Yes
59,614,France,Male,nan,4.000000,40685.920000,1,1,1,46775.280000,No
92,524,France,Female,nan,10.000000,0.000000,2,1,0,109614.570000,No
118,524,Germany,nan,31.000000,8.000000,107818.630000,1,1,0,199725.390000,Yes
137,637,France,Male,nan,2.000000,133463.100000,1,0,1,93165.340000,No
173,578,France,nan,30.000000,4.000000,169462.090000,1,1,0,112187.110000,No
195,807,France,Male,nan,3.000000,174790.150000,1,1,1,600.360000,No


Gender: Missing data should be imputed with the same distrubution of Male/Female of the
rest of the data set

In [9]:
count_missing_gender = dataset.loc[:, 'Gender'].isna().sum()
count_missing_gender

count = dataset.loc[:, 'Gender'].value_counts(normalize=True)
count


gender_amount = (count * count_missing_gender).round().astype(int)
gender_amount

gender_colum_name = gender_amount.name

gener_data = np.repeat(gender_amount.keys().values, gender_amount.values)
gener_data


isna = dataset.loc[:, 'Gender'].isna()
missing_index = dataset[isna].index
missing_index

gender_update = pd.Series(
    data=gener_data, index=missing_index, name=gender_colum_name)

dataset.update(gender_update)
dataset

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42.0,2.0,0.00,1,1,1,101348.88,Yes
1,608,Spain,Female,41.0,1.0,83807.86,1,0,1,112542.58,No
2,502,France,Female,42.0,8.0,159660.80,3,1,0,113931.57,Yes
3,699,France,Female,39.0,1.0,0.00,2,0,0,93826.63,No
4,850,Spain,Female,43.0,2.0,125510.82,1,1,1,79084.10,No
...,...,...,...,...,...,...,...,...,...,...,...
995,610,Spain,Male,29.0,9.0,0.00,3,0,1,83912.24,No
996,811,Spain,Male,44.0,3.0,0.00,2,0,1,78439.73,No
997,587,Spain,Male,62.0,7.0,121286.27,1,0,1,6776.92,No
998,811,Germany,Female,28.0,4.0,167738.82,2,1,1,9903.42,No


 Age: Impute with mean

In [10]:
age_tenure_median = dataset.loc[:, ['Age', 'Tenure']].mean()

age_tenure_median_nona = dataset.loc[:, ['Age', 'Tenure']].fillna(
    age_tenure_median).astype(int)


dataset[['Age', 'Tenure']] = age_tenure_median_nona


 Geography: Transform it with Pandas

In [11]:
dataset = pd.get_dummies(dataset, prefix='Geography', columns=['Geography'], dtype=int)
dataset

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,Yes,1,0,0
1,608,Female,41,1,83807.86,1,0,1,112542.58,No,0,0,1
2,502,Female,42,8,159660.80,3,1,0,113931.57,Yes,1,0,0
3,699,Female,39,1,0.00,2,0,0,93826.63,No,1,0,0
4,850,Female,43,2,125510.82,1,1,1,79084.10,No,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,610,Male,29,9,0.00,3,0,1,83912.24,No,0,0,1
996,811,Male,44,3,0.00,2,0,1,78439.73,No,0,0,1
997,587,Male,62,7,121286.27,1,0,1,6776.92,No,0,0,1
998,811,Female,28,4,167738.82,2,1,1,9903.42,No,0,1,0


Gender & Exited: Transform it with OrdinalEncoder instead of LabelEncoder

In [12]:
enc = OrdinalEncoder()
gender_exited_ordinal = enc.fit_transform(dataset.loc[:,['Gender', 'Exited']]).astype(int)

dataset[['Gender', 'Exited']] = gender_exited_ordinal
gender_exited_ordinal


array([[0, 1],
       [0, 0],
       [0, 1],
       ...,
       [1, 0],
       [0, 0],
       [0, 0]])

Split the data-set into the Training- and Test-set (80/20)

In [13]:
train, test = train_test_split(dataset, test_size=0.2, random_state=0)
display(train)
display(test)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
613,667,1,38,1,87202.38,1,1,1,77866.91,0,0,0,1
38,850,1,36,7,0.00,1,1,1,40812.90,0,1,0,0
662,678,1,31,2,0.00,2,1,1,58803.28,0,1,0,0
896,718,1,35,8,0.00,2,1,0,94820.85,0,1,0,0
490,591,1,29,1,97541.24,1,1,1,196356.17,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,505,1,49,7,80001.23,1,0,0,135180.11,0,1,0,0
195,807,1,39,3,174790.15,1,1,1,600.36,0,1,0,0
639,785,0,36,4,135438.40,1,0,0,190627.01,0,1,0,0
568,737,1,31,5,121192.22,2,1,1,74890.58,0,0,1,0


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
757,736,0,28,2,0.00,2,1,1,117431.10,0,0,0,1
635,696,0,51,5,0.00,2,1,0,55022.43,0,0,0,1
303,712,0,33,6,0.00,2,1,1,190686.16,0,1,0,0
756,664,0,36,2,127160.78,2,1,0,78140.75,0,0,1,0
46,829,0,27,9,112045.67,1,1,1,119708.21,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,549,0,39,7,0.00,1,0,0,81259.25,1,0,0,1
964,683,0,43,4,115888.04,1,1,1,117349.19,1,0,1,0
874,645,0,38,5,101430.30,2,0,1,4400.32,0,1,0,0
883,643,0,45,4,45144.43,1,1,0,60917.24,1,1,0,0


Apply Feature Scaling 
- Values wich can me put in a bucket sould be normalized
- Valued wich differ a lot should be standardized

Age, NumOfProducts => normalized
Balance, EstimatedSalary => standardized

In [14]:
min_max_scaler = MinMaxScaler()
train[['Age', 'NumOfProducts']] = min_max_scaler.fit_transform(
    train.loc[:, ['Age', 'NumOfProducts']])

test[['Age', 'NumOfProducts']] = min_max_scaler.fit_transform(
    test.loc[:, ['Age', 'NumOfProducts']])

display(train)
display(test)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
613,667,1,0.312500,1,87202.38,0.000000,1,1,77866.91,0,0,0,1
38,850,1,0.281250,7,0.00,0.000000,1,1,40812.90,0,1,0,0
662,678,1,0.203125,2,0.00,0.333333,1,1,58803.28,0,1,0,0
896,718,1,0.265625,8,0.00,0.333333,1,0,94820.85,0,1,0,0
490,591,1,0.171875,1,97541.24,0.000000,1,1,196356.17,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,505,1,0.484375,7,80001.23,0.000000,0,0,135180.11,0,1,0,0
195,807,1,0.328125,3,174790.15,0.000000,1,1,600.36,0,1,0,0
639,785,0,0.281250,4,135438.40,0.000000,0,0,190627.01,0,1,0,0
568,737,1,0.203125,5,121192.22,0.333333,1,1,74890.58,0,0,1,0


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
757,736,0,0.142857,2,0.00,0.5,1,1,117431.10,0,0,0,1
635,696,0,0.612245,5,0.00,0.5,1,0,55022.43,0,0,0,1
303,712,0,0.244898,6,0.00,0.5,1,1,190686.16,0,1,0,0
756,664,0,0.306122,2,127160.78,0.5,1,0,78140.75,0,0,1,0
46,829,0,0.122449,9,112045.67,0.0,1,1,119708.21,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,549,0,0.367347,7,0.00,0.0,0,0,81259.25,1,0,0,1
964,683,0,0.448980,4,115888.04,0.0,1,1,117349.19,1,0,1,0
874,645,0,0.346939,5,101430.30,0.5,0,1,4400.32,0,1,0,0
883,643,0,0.489796,4,45144.43,0.0,1,0,60917.24,1,1,0,0


In [15]:
standard_scaler = StandardScaler()
train[['Balance', 'EstimatedSalary']] = standard_scaler.fit_transform(
    train.loc[:, ['Balance', 'EstimatedSalary']])

test[['Balance', 'EstimatedSalary']] = standard_scaler.fit_transform(
    test.loc[:, ['Balance', 'EstimatedSalary']])

display(train)
display(test)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
613,667,1,0.312500,1,0.154824,0.000000,1,1,-0.365701,0,0,0,1
38,850,1,0.281250,7,-1.221929,0.000000,1,1,-1.010701,0,1,0,0
662,678,1,0.203125,2,-1.221929,0.333333,1,1,-0.697542,0,1,0,0
896,718,1,0.265625,8,-1.221929,0.333333,1,0,-0.070584,0,1,0,0
490,591,1,0.171875,1,0.318054,0.000000,1,1,1.696843,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,505,1,0.484375,7,0.041132,0.000000,0,0,0.631950,0,1,0,0
195,807,1,0.328125,3,1.537662,0.000000,1,1,-1.710682,0,1,0,0
639,785,0,0.281250,4,0.916376,0.000000,0,0,1.597116,0,1,0,0
568,737,1,0.203125,5,0.691457,0.333333,1,1,-0.417510,0,0,1,0


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
757,736,0,0.142857,2,-1.360680,0.5,1,1,0.352457,0,0,0,1
635,696,0,0.612245,5,-1.360680,0.5,1,0,-0.757333,0,0,0,1
303,712,0,0.244898,6,-1.360680,0.5,1,1,1.655124,0,1,0,0
756,664,0,0.306122,2,0.763239,0.5,1,0,-0.346228,0,0,1,0
46,829,0,0.122449,9,0.510777,0.0,1,1,0.392950,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,549,0,0.367347,7,-1.360680,0.0,0,0,-0.290773,1,0,0,1
964,683,0,0.448980,4,0.574954,0.0,1,1,0.351000,1,0,1,0
874,645,0,0.346939,5,0.333472,0.5,0,1,-1.657526,0,1,0,0
883,643,0,0.489796,4,-0.606649,0.0,1,0,-0.652508,1,1,0,0


describe normalization

In [16]:

def desc_normalization(title: str, df):
    describe_data = df[['Age', 'NumOfProducts']].describe(percentiles=[]).loc[['min', 'max']]
    display(title)
    display(describe_data)

desc_normalization('train', train)
desc_normalization('test', test)


'train'

,Age,NumOfProducts
min,0.0,0.0
max,1.0,1.0


'test'

,Age,NumOfProducts
min,0.0,0.0
max,1.0,1.0


describe standardization

In [17]:
def desc_standardization(title: str, df):
    describe_data = df[['Balance', 'EstimatedSalary']].describe(
        percentiles=[]).loc[['mean', 'std']]
    display(title)
    display(describe_data)


desc_standardization('train', train)
desc_standardization('test', test)


'train'

,Balance,EstimatedSalary
mean,-1.256010e-16,6.728624e-17
std,1.000632e+00,1.000632e+00


'test'

,Balance,EstimatedSalary
mean,-7.289343e-18,-2.601735e-16
std,1.002535e+00,1.002535e+00


In [18]:
display(train)
display(train.dtypes)
display(test)
display(test.dtypes)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
613,667,1,0.312500,1,0.154824,0.000000,1,1,-0.365701,0,0,0,1
38,850,1,0.281250,7,-1.221929,0.000000,1,1,-1.010701,0,1,0,0
662,678,1,0.203125,2,-1.221929,0.333333,1,1,-0.697542,0,1,0,0
896,718,1,0.265625,8,-1.221929,0.333333,1,0,-0.070584,0,1,0,0
490,591,1,0.171875,1,0.318054,0.000000,1,1,1.696843,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,505,1,0.484375,7,0.041132,0.000000,0,0,0.631950,0,1,0,0
195,807,1,0.328125,3,1.537662,0.000000,1,1,-1.710682,0,1,0,0
639,785,0,0.281250,4,0.916376,0.000000,0,0,1.597116,0,1,0,0
568,737,1,0.203125,5,0.691457,0.333333,1,1,-0.417510,0,0,1,0


CreditScore            int64
Gender                 int64
Age                  float64
Tenure                 int64
Balance              float64
NumOfProducts        float64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary      float64
Exited                 int64
Geography_France       int64
Geography_Germany      int64
Geography_Spain        int64
dtype: object

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
757,736,0,0.142857,2,-1.360680,0.5,1,1,0.352457,0,0,0,1
635,696,0,0.612245,5,-1.360680,0.5,1,0,-0.757333,0,0,0,1
303,712,0,0.244898,6,-1.360680,0.5,1,1,1.655124,0,1,0,0
756,664,0,0.306122,2,0.763239,0.5,1,0,-0.346228,0,0,1,0
46,829,0,0.122449,9,0.510777,0.0,1,1,0.392950,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,549,0,0.367347,7,-1.360680,0.0,0,0,-0.290773,1,0,0,1
964,683,0,0.448980,4,0.574954,0.0,1,1,0.351000,1,0,1,0
874,645,0,0.346939,5,0.333472,0.5,0,1,-1.657526,0,1,0,0
883,643,0,0.489796,4,-0.606649,0.0,1,0,-0.652508,1,1,0,0


CreditScore            int64
Gender                 int64
Age                  float64
Tenure                 int64
Balance              float64
NumOfProducts        float64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary      float64
Exited                 int64
Geography_France       int64
Geography_Germany      int64
Geography_Spain        int64
dtype: object

save data

In [19]:
train.to_csv('result/train.csv', sep=';', index=False, decimal=',')
test.to_csv('result/test.csv', sep=';', index=False, decimal=',')
